In [17]:
import gzip
import numpy as np
from collections import defaultdict


def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

def readCSV(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        yield l.strip().split(',')


train_data = np.array(list(readCSV('train_Interactions.csv.gz')))

test_data = []
predictions = open("predictions_Read.csv", 'w')
for l in open("pairs_Read.csv"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    test_data.append(l.strip().split(','))


book_counts = defaultdict(int)
for entry in train_data:
    bookID = entry[1] 
    book_counts[bookID] += 1

total_read = len(train_data)
threshold = total_read * 0.69
most_popular = []
for bookID, count in book_counts.items():
    most_popular.append((count, bookID))
most_popular.sort(reverse=True)

popular_books = set()
count = 0
for ic, bookID in most_popular:
    count += ic
    popular_books.add(bookID)
    if count > threshold:
        break

for entry in test_data:
    userID, bookID = entry
    prediction = 1 if bookID in popular_books else 0
    predictions.write(f"{userID},{bookID},{prediction}\n")

predictions.close()


In [ ]:


train_data = list(readCSV('train_Interactions.csv.gz'))
predictions = open("predictions_Rating.csv", 'w')
for l in open("pairs_Rating.csv"):
    if l.startswith("userID"):
        predictions.write(l)
        continue

test_data = []
with open('pairs_Rating.csv', 'r') as f:
    for line in f:
        if line.startswith("userID"):
            continue
        test_data.append(line.strip().split(','))


train_data = np.array(train_data)
user_ids = train_data[:, 0]
book_ids = train_data[:, 1]
ratings = train_data[:, 2].astype(float)

global_avg = ratings.mean()


user_avg = defaultdict(float)
book_avg = defaultdict(float)
user_counts = defaultdict(int)
book_counts = defaultdict(int)


for user, book, rating in zip(user_ids, book_ids, ratings):
    user_avg[user] += rating
    book_avg[book] += rating
    user_counts[user] += 1
    book_counts[book] += 1

for user in user_avg:
    user_avg[user] /= user_counts[user]

for book in book_avg:
    book_avg[book] /= book_counts[book]

for user, book in test_data:
   
    user_weight = min(user_counts.get(user, 0) / 5, 1)
    book_weight = min(book_counts.get(book, 0) / 31, 1)
    
    if user in user_avg and book in book_avg:
        pred = (user_weight * user_avg[user] + book_weight * book_avg[book]) / (user_weight + book_weight)
    elif user in user_avg:
        pred = user_avg[user]
    elif book in book_avg:
        pred = book_avg[book]
    else:
        pred = global_avg

    predictions.write(f"{user},{book},{pred}\n")
    print(user, book, pred)

predictions.close()



u11538900 b07888199 4.385714285714286
u97036836 b27008357 4.053518440016957
u51926222 b42417554 4.156028368794326
u00942187 b40172367 3.423255813953488
u22105217 b64130684 3.5871980676328503
u24244364 b22207461 4.053571428571429
u37717598 b00366117 3.6455653351387944
u18606162 b50550215 3.957044673539519
u70148748 b73826999 3.8145299145299143
u96964690 b11764474 4.094246031746032
u28527596 b14854557 3.362165178571429
u37577857 b97382810 0.8446428571428573
u49707395 b09887883 3.1166666666666667
u38055540 b91045742 3.550965250965251
u22950188 b58549702 3.772310824567357
u65645818 b16299278 3.4413793103448276
u94398205 b40722160 4.313725490196078
u05399231 b55294529 3.1445221445221447
u31447336 b45146512 3.6686046511627906
u74942276 b01028561 3.6919642857142856
u25793793 b47683735 4.153846153846154
u42099743 b15118141 3.291803278688524
u98715757 b93074227 3.7301587301587302
u51521854 b80742104 3.447916666666667
u56535163 b29795443 4.161392405063291
u15255766 b49695798 3.747792494481236
u5